## Implement `LinkedListDeque` class

The deque has to be able to take any type of data. In this case, we use generics `<T>`. In this implementation, we'll use circular sentinel.

In [ ]:
public class LinkedListDeque<T>{
    ... // Everything goes inside here!
}

### `Node` class

A `Node` contains the following:
* `prev` = a pointer pointing to the previous Node
* `item` = the item contained in a node, can be of any type
* `next` = a pointer pointing to the next Node.

1. It is a good idea to set the `Node` class `private` rather than `public` to ensure that a `Node` can't be modified by anything other than the provided methods.

2. And don't forget to implement the constructor for a `Node`!

In [ ]:
private class Node{
    public Node prev; // A pointer that points to the previous Node
    public T item; // The item contained in a node
    public Node next; // A pointer that points to the next Node
    
    // Node object constructor
    public Node (Node p, T i, Node n) {
        prev = p;
        item = i;
        next= n;
    }
}

### `LinkedListDeque` Instance Variables

Every `LinkedListDeque` has these 2 things:
* `sentinel` Node
* `size`, which is the length of the linked list

Similar to `Node` class, it is a good idea to set these 2 things as `private` so that it can't be modified without using the provided methods.

In [ ]:
private Node sentinel;
private int size;

### `LinkedListDeque` Empty Constructor

<img src = 'circle.png' width = 700/>

As shown by the design above, in the initials state we want the `sentinel` to be circular:
* `sentinel.next` is pointing back at `sentinel`
* `sentinel.prev` is pointing back at `sentinel`

However, BEWARE not to set the `sentinel` as specified above on the same time,

In [ ]:
sentinel = new Node(sentinel, null, sentinel); // THIS IS A MISTAKE!

Initially, we haven't assigned the `sentinel` to anything (`sentinel` is still `null`). Therefore, when we execute the line above, the `sentinel.prev` and `sentinel.next` will be set to `null` instead of `sentinel`!

Instead, we should set the `sentinel.next` and `sentinel.prev` after we are done initializing the `sentinel`.

In [ ]:
public LinkedListDeque(){
    sentinel = new Node(null, null, null);
    sentinel.next = sentinel;
    sentinel.prev = sentinel;
    size = 0;
}

Note above that we set the `sentinel`'s `item` to `null`. Recall that the `item` within `sentinel` doesn't matter. `null` is chosen because `null` can be used for any data types (remember we are using generics)!

### `addFirst`

One flawed understanding is to think that we can simply "change" the `Node` after the `sentinel` to a new `Node`.

In [ ]:
public void addFirst(T item) {
    sentinel.next = new Node(sentinel, item, sentinel.next);
    size++; // Increment the size
}

Let's say we have the following Deque:

`Sentinel <--> 3 <--> 4`

...and we want to `addFirst(2)`. By only setting `sentinel.next` as above, we disconnected the circular link.

* `Sentinel <--> 2 -> 3`
* `Sentinel <- 3 <--> 4`

The `.prev` of the `Node` containing 3 is still set to the `sentinel`! 

Therefore, we want to set `3`'s `prev` as well! Instead of setting the `sentinel.next` as the new `Node` right away, it makes more sense to separately create the new `Node` and assign the `3`'s `prev` and `sentinel.next` with the new `Node`.

In [2]:
public void addFirst(T item) {
    Node a = new Node(sentinel, item, sentinel.next);
    sentinel.next.prev = a; // Set 3's prev to the new Node
    sentinel.next = a; // Set sentinel.next to the new Node
    size++; // Increment size
}

SyntaxError: invalid syntax (<ipython-input-2-498168ba42d1>, line 1)

### `addLast`

Similar to `addFirst`, we don't want to only set `sentinel.prev`. We want to set both `sentinel.prev.next` and `sentinel.prev`.

In [ ]:
public void addLast(T item){
    Node a = new Node(sentinel.prev, item, sentinel);
    sentinel.prev.next = a;
    sentinel.prev = a;
    size++;
}

### `isEmpty()`

Simply check whether the `size` instance attribute is 0. If yes, then the deque is considered empty.

In [ ]:
public boolean isEmpty(){
    return (size == 0);
}

### `size()`

Simply return the value within the instance variable `size`.

In [ ]:
public int size() {
    return size;
}

### `removeFirst()`

The base case is that if the deque is empty (`size` is 0), then we just return `null`.

Otherwise:
1. Extract the `item` that's contained by the `Node` next to the `sentinel`
2. Adjust the `sentinel.next.prev`
3. Adjust the `sentinel.next`
4. Decrement `size`
5. Return the `item` that was extracted in step #1

Once again, a common mistake is to adjust only the `sentinel.next` without adjusting the `sentinel.next.prev` pointer.

In [ ]:
public T removeFirst(){
    if (isEmpty()) return null;
    else {
        T result = sentinel.next.item;
        Node a = new Node(sentinel, sentinel.next.next.item, sentinel.next.next.next);
        sentinel.next.prev = a;
        sentinel.next = a;
        size--;
        return result;
    }
}

### `removeLast()`

Similar to `removeFirst()`, the differences are in step 1-3:

1. Extract the `item` that's contained bt the `Node` previous of the `sentinel`
2. Adjust the `sentinel.prev.next`
3. Adjust the `sentinel.prev`

A common mistake here is to adjust only `sentinel.prev` without adjusting the `sentinel.prev.next` pointer.

In [3]:
public T removeFirst(){
    if (isEmpty()) return null;
    else {
        T result = sentinel.prev.item;
        Node a = new Node(sentinel.prev.prev.prev, sentinel.prev.prev.item, sentinel);
        sentinel.prev.next = a;
        sentinel.prev = a;
        size--;
        return result;
    }
}

SyntaxError: invalid syntax (<ipython-input-3-fd0831dd867d>, line 1)

### `get`

Conceptually, we can optimize the `get` function with the idea that: If the desired element is located...
* ...after the midpoint of the linked list, then cycle through the elements backwards
* ...before the midpoint of the linked list, then cycle forward
* ... at the midpoint of the linked list, it doesn't matter if we cycle backwards or forward.

#### Example #1
We have the following linked list:

`[Sentinel, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]`

size = 16 (even number)
* index 6 -> forward 7 steps, backwards 10 steps
* index 7 -> forward 8 steps, backwards 9 steps
* index 8 -> forward 9 steps, backwards 8 steps
* index 9 -> forward 10 steps, backwards 7 steps

#### Example #2

`[Sentinel, 0, 1, 2, 3, 4, 5, 6, 7, 8]`

size = 9
* index 3 -> forward 4 steps, backwards 6 steps
* index 4 -> forward 5 steps, backwards 5 steps
* index 5 -> forward 6 steps, backwards 4 steps

It doesn't matter if the `size` is odd or even. We can implement the method so that if the index is `half of the size or greater`, cycle through the linked list backwards (`9 / 2` is `4` anyway).

In [ ]:
public T get(int index) {
    Node pointer = sentinel; // Start a pointer at the sentinel
    if (index >= size / 2){ // If the index is equal to greater than half of the size, iterate backwards
        for (int i = size - index; i > 0; i--) pointer = pointer.prev;
        return pointer.item;
    } else { // Otherwise, iterate forward
        for (int i = size - index; i > 0; i--) pointer = pointer.next;
        return pointer.item;
    }
}

Notice in the `for` loop that we iterates with `int i = size - index` decrementing as long as `i` is greater than 0. In the example of:

`[Sentinel, 0, 1, 2, 3, 4, 5, 6, 7, 8]`

...`size = 9`, `index` = `4`, then the `for` loop starts with `i = 5`. This means cycle backwards 5 steps from the `sentinel`. We'll see that we ended up with the element `4`, which is the desired element!

### `printDeque()`

The idea is to initially set a pointer to the `Node` next to `sentinel` (`sentinel.next`), then do a `while` loop:

* `While:`We know the `item` within the `sentinel` is `null`, so we cycle through the elements and stops when we encounter an `item` that is `null`, which indicates that we've cycled all the way back to the `sentinel`
* Within the `while` loop, do the following:
    * Print the `pointer.item`, but convert it to `String` beforehand since it's not guaranteed that the `item` is a `String`
    * Print a space
    * Cycle through the next `Node`
    
At the end of the `while` loop, we print a new line.

In [ ]:
public void printDeque(){
    Node pointer = sentinel.next; // Set the pointer to sentinel.next
    while (pointer.item != null) { // As long as the item is not null
        System.out.print(pointer.item.toString()); // Convert the item to String then print it
        System.out.print(" "); // Prints a space
        pointer = pointer.next; // Cycle forward to the next Node
    }
    System.out.println(""); // Once done with the loop, print a new line
}

### `LinkedListDeque(LinkedListDeque other)`

Since this method is a constructor, we need to initiate the circular sentinel structure as done in the empty constructor. Note that we can't just call the the empty constructor within this method; we will get an error if we do this!

In [ ]:
public LinkedListDeque(LinkedListDeque other) {
    LinkedListDeque(); // WILL GIVE AN ERROR!
    ...
}

Instead, we manually initiate the circular sentinel structure.

In [ ]:
public LinkedListDeque(LinkedListDeque other) {
    sentinel = new Node(null, null, null);
    sentinel.next = sentinel;
    sentinel.prev = sentinel;
    size = 0;
    ...
}

...and all that's left is to obtain each of the elements in `other`. We can use the `get` method to obtain elements from `other`, which means we would need to have an integer iterator for the index argument.

In [ ]:
for (int i = 0; i < other.size; i++){
    addLast((T) other.get(i)); // Use casting!
}